In [1]:
import numpy as np

from preprocessing import obtain_chembl_data,combine_chembl_papyrus_mutants,merge_chembl_papyrus_mutants
from annotation import *

### Annotation workflow step by step

In [2]:
# 1. Get ChEMBL data using ChEMBl downloader
chembl_data = obtain_chembl_data(chembl_version='31', chunksize= 100_000)
chembl_data

C:\Users\gorostiolam\AppData\Local\Continuum\anaconda3\envs\mutants-in-pcm\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (4,6,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  


,description,assay_id,variant_id,assay_chembl_id,assay_organism,year,abstract,mutation,activity_id,pchembl_value,standard_type,activity_comment,chembl_id,canonical_smiles,accession,sequence,organism
0,In vivo inhibitory activity against human Hepa...,83907,NaN,CHEMBL872937,NaN,2004.0,"A novel class of 2,3-dihydro-1,3-dioxo-1H-isoi...",NaN,31864,5.60,IC50,NaN,CHEMBL324340,Cc1ccc2oc(-c3cccc(N4C(=O)c5ccc(C(=O)O)cc5C4=O)...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,Homo sapiens
1,In vivo inhibitory activity against human Hepa...,83907,NaN,CHEMBL872937,NaN,2004.0,"A novel class of 2,3-dihydro-1,3-dioxo-1H-isoi...",NaN,31866,5.05,IC50,NaN,CHEMBL109600,COc1ccccc1-c1ccc2oc(-c3ccc(OC)c(N4C(=O)c5ccc(C...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,Homo sapiens
2,Inhibitory activity against Palmitoyl-CoA oxid...,154606,NaN,CHEMBL760688,NaN,2004.0,New inhibitors of palmitoylCoA oxidation were ...,NaN,31868,5.40,IC50,NaN,CHEMBL357278,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4ccc(Cl)c(C(...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,Rattus norvegicus
3,Inhibitory activity against Palmitoyl-CoA oxid...,154606,NaN,CHEMBL760688,NaN,2004.0,New inhibitors of palmitoylCoA oxidation were ...,NaN,31870,4.77,IC50,NaN,CHEMBL357119,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)NCCc4ccccc4)CC...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,Rattus norvegicus
4,Inhibitory activity against Palmitoyl-CoA oxid...,154606,NaN,CHEMBL760688,NaN,2004.0,New inhibitors of palmitoylCoA oxidation were ...,NaN,31872,6.75,IC50,NaN,CHEMBL152968,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccc...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,Rattus norvegicus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270179,KinomeScan assay: inhibition of LATS2,2136225,NaN,CHEMBL4879778,Homo sapiens,2021.0,DiscoverX KinomeScan assays for SGC Frankfurt ...,NaN,23347581,7.26,Kd,Not Active,CHEMBL4538174,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,Homo sapiens
5270180,KinomeScan assay: inhibition of ROCK1,2136385,NaN,CHEMBL4879938,Homo sapiens,2021.0,DiscoverX KinomeScan assays for SGC Frankfurt ...,NaN,23347583,7.01,Kd,Not Active,CHEMBL4538174,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,Homo sapiens
5270181,KinomeScan assay: inhibition of ROCK2,2136386,NaN,CHEMBL4879939,Homo sapiens,2021.0,DiscoverX KinomeScan assays for SGC Frankfurt ...,NaN,23347585,7.09,Kd,Not Active,CHEMBL4538174,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,Homo sapiens
5270182,KinomeScan assay: inhibition of TGFBR2,2136430,NaN,CHEMBL4879983,Homo sapiens,2021.0,DiscoverX KinomeScan assays for SGC Frankfurt ...,NaN,23347587,7.27,Kd,Not Active,CHEMBL4538174,C[C@@H](NC(=O)Nc1cc2[nH]ncc2c(CO)n1)c1ccccc1,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,Homo sapiens


In [3]:
# 2. Get unique assays from ChEMBL
chembl_assays = filter_assay_data(chembl_data)
chembl_assays

,assay_id,description,accession,sequence,mutation
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN
2,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN
5,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN
6,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN
7,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN
...,...,...,...,...,...
5270179,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN
5270180,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN
5270181,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN
5270182,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN


In [4]:
# 3. Extract mutation-looking ReGex patterns from assay descriptions and store them in column aa_change
# Note: column mutation contains annotated ChEMBL variants
chembl_assays_extracted = extract_aa_change(chembl_assays)
chembl_assays_extracted

,assay_id,description,accession,sequence,mutation,aa_change
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN,[]
2,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN,[]
5,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN,[]
6,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN,[]
7,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN,[]
...,...,...,...,...,...,...
5270179,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN,[]
5270180,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN,[]
5270181,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN,[]
5270182,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN,[]


In [5]:
# Check how many mutations were extracted with ReGex expressions
chembl_assays_extracted[chembl_assays_extracted.apply(lambda x: len(x['aa_change']) > 0, axis=1)]

,assay_id,description,accession,sequence,mutation,aa_change
178,59914,Binding affinity to cloned human Dopamine rece...,P14416,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,NaN,[A9L]
293,106820,Agonist activity at human melanocortin recepto...,P32245,MVNSTHRGMHTSLHLWNRSSYRLHSNASESLGKGYSDGGCYEQLFV...,NaN,[C4R]
294,106816,Agonistic activity against human melanocortin ...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R]
296,106815,Agonist activity at human melanocortin recepto...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R]
499,2560,Ability to stimulate phosphoinositide hydrolys...,P14842,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,NaN,[H3T]
...,...,...,...,...,...,...
5270146,2135964,Inhibition of human PI3K p110beta/p85alpha by ...,P42338,MCFSFIMPPAMADILDIWAVDSQIASDGSIPVDFLLPTGIYIQLEV...,NaN,[I3K]
5270149,2135965,Inhibition of human PI3K p110delta/p85alpha by...,P27986,MSAEGYQYRALYDYKKEREEDIDLHLGDILTVNKGSLVALGFSDGQ...,NaN,[I3K]
5270150,2135965,Inhibition of human PI3K p110delta/p85alpha by...,O00329,MPPGVDCPMEFWTKEENQSVVVDFLLPTGVYLNFPVSRNANLSTIK...,NaN,[I3K]
5270153,2135966,Inhibition of human PI3Kgamma by HTRF assay,P48736,MELENYKQPVVLREDNCRRRRRMKPRSAAASLSSMELIPIEFVLPT...,NaN,[I3K]


In [6]:
# 4. Visualize exceptions defined by matches with cell lines, targets and genes
# This step is included in validate_aa_change()
exceptions = define_aa_change_exceptions(chembl_assays_extracted, chembl_version='31')
print_exceptions =  exceptions[exceptions.apply(lambda x: len(x['exception_reasons']) > 0, axis=1)]
print_exceptions

,assay_id,description,accession,sequence,mutation,aa_change,exception_flags,exception_reasons
84,59914,Binding affinity to cloned human Dopamine rece...,P14416,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,NaN,[A9L],[True],[assay_cell_type]
174,106820,Agonist activity at human melanocortin recepto...,P32245,MVNSTHRGMHTSLHLWNRSSYRLHSNASESLGKGYSDGGCYEQLFV...,NaN,[C4R],[True],[component_synonym]
175,106816,Agonistic activity against human melanocortin ...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R],[True],[component_synonym]
176,106815,Agonist activity at human melanocortin recepto...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R],[True],[component_synonym]
312,2560,Ability to stimulate phosphoinositide hydrolys...,P14842,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,NaN,[H3T],[True],[cell_name]
...,...,...,...,...,...,...,...,...
376205,2135964,Inhibition of human PI3K p110beta/p85alpha by ...,P42338,MCFSFIMPPAMADILDIWAVDSQIASDGSIPVDFLLPTGIYIQLEV...,NaN,[I3K],[True],[pref_name]
376206,2135965,Inhibition of human PI3K p110delta/p85alpha by...,P27986,MSAEGYQYRALYDYKKEREEDIDLHLGDILTVNKGSLVALGFSDGQ...,NaN,[I3K],[True],[component_synonym]
376207,2135965,Inhibition of human PI3K p110delta/p85alpha by...,O00329,MPPGVDCPMEFWTKEENQSVVVDFLLPTGVYLNFPVSRNANLSTIK...,NaN,[I3K],[True],[component_synonym]
376208,2135966,Inhibition of human PI3Kgamma by HTRF assay,P48736,MELENYKQPVVLREDNCRRRRRMKPRSAAASLSSMELIPIEFVLPT...,NaN,[I3K],[True],[component_synonym]


In [7]:
# Check how many extracted mutations had exceptions defined
exceptions[exceptions.apply(lambda x: True in x['exception_flags'], axis=1) & exceptions.apply(lambda x: len(x['aa_change']) > 0, axis=1)]

,assay_id,description,accession,sequence,mutation,aa_change,exception_flags,exception_reasons
84,59914,Binding affinity to cloned human Dopamine rece...,P14416,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,NaN,[A9L],[True],[assay_cell_type]
174,106820,Agonist activity at human melanocortin recepto...,P32245,MVNSTHRGMHTSLHLWNRSSYRLHSNASESLGKGYSDGGCYEQLFV...,NaN,[C4R],[True],[component_synonym]
175,106816,Agonistic activity against human melanocortin ...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R],[True],[component_synonym]
176,106815,Agonist activity at human melanocortin recepto...,Q01726,MAVQGSQRRLLGSLNSTPTAIPQLGLAANQTGARCLEVSISDGLFL...,NaN,[C1R],[True],[component_synonym]
312,2560,Ability to stimulate phosphoinositide hydrolys...,P14842,MEILCEDNISLSSIPNSLMQLGDGPRLYHNDFNSRDANTSEASNWT...,NaN,[H3T],[True],[cell_name]
...,...,...,...,...,...,...,...,...
376205,2135964,Inhibition of human PI3K p110beta/p85alpha by ...,P42338,MCFSFIMPPAMADILDIWAVDSQIASDGSIPVDFLLPTGIYIQLEV...,NaN,[I3K],[True],[pref_name]
376206,2135965,Inhibition of human PI3K p110delta/p85alpha by...,P27986,MSAEGYQYRALYDYKKEREEDIDLHLGDILTVNKGSLVALGFSDGQ...,NaN,[I3K],[True],[component_synonym]
376207,2135965,Inhibition of human PI3K p110delta/p85alpha by...,O00329,MPPGVDCPMEFWTKEENQSVVVDFLLPTGVYLNFPVSRNANLSTIK...,NaN,[I3K],[True],[component_synonym]
376208,2135966,Inhibition of human PI3Kgamma by HTRF assay,P48736,MELENYKQPVVLREDNCRRRRRMKPRSAAASLSSMELIPIEFVLPT...,NaN,[I3K],[True],[component_synonym]


In [8]:
# 5. Validate extracted mutations based on the previously defined exceptions and their WT sequence
chembl_assays_validated = validate_aa_change(chembl_assays_extracted, chembl_version='31')
chembl_assays_validated

,assay_id,description,accession,sequence,mutation,aa_change,mutants
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN,[],[]
1,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN,[],[]
2,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN,[],[]
3,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN,[],[]
4,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN,[],[]
...,...,...,...,...,...,...,...
376228,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN,[],[]
376229,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN,[],[]
376230,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN,[],[]
376231,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN,[],[]


In [9]:
# 6. Based on the validated mutants, create Papyrus-like identifiers
chembl_assays_annotated = create_papyrus_columns(chembl_assays_validated)
chembl_assays_annotated

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN,[],[],Q9Y251_WT,WT
1,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN,[],[],P07872_WT,WT
2,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN,[],[],P05177_WT,WT
3,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN,[],[],P08684_WT,WT
4,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN,[],[],P11712_WT,WT
...,...,...,...,...,...,...,...,...,...
376228,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN,[],[],Q9NRM7_WT,WT
376229,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN,[],[],Q13464_WT,WT
376230,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN,[],[],O75116_WT,WT
376231,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN,[],[],P37173_WT,WT


In [ ]:
# 7. Map newly annotated assays to the ChEMBL bioactivity dataset
chembl_bioactivity_dataset = map_activity_mutations(chembl_data, chembl_assays_annotated)
chembl_bioactivity_dataset

In [ ]:
# Check how many bioactivity datapoints in ChEMBL are tested in mutants
chembl_bioactivity_dataset[~chembl_bioactivity_dataset['target_id'].str.contains('WT')]

In [ ]:
# 8. Concatenate newly annotated ChEMBL bioactivity data with non-ChEMBL part of Papyrus (only targets with at least
# one annotated variant)
chembl_papyrus_bioactivity_dataset = combine_chembl_papyrus_mutants('31', '05.5', 'nostereo', 1_000_000)
chembl_papyrus_bioactivity_dataset

In [ ]:
# 9. Merge ChEMBL and Papyrus data by calculating mean pChEMBL values (only targets with at least one annotated variant)
combined_dataset = merge_chembl_papyrus_mutants('31', '05.5', 'nostereo', 1_000_000)
combined_dataset

In [ ]:
# Check how many bioactivity datapoints in ChEMBL + Papyrus are tested in mutants
combined_dataset[~combined_dataset['target_id'].str.contains('WT')]

In [ ]:
# Check the distribution of per source of mutant bioactivity datapoints
combined_dataset[~combined_dataset['target_id'].str.contains('WT')].groupby(['source'])['pchembl_value_Mean'].count()

In [ ]:
# Check how many bioactivity datapoints in ChEMBL + Papyrus are tested in mutants (BEFORE MERGING)
chembl_papyrus_bioactivity_dataset[~chembl_papyrus_bioactivity_dataset['target_id'].str.contains('WT')]

In [ ]:
# Check the distribution of per source of mutant bioactivity datapoints (BEFORE MERGING)
chembl_papyrus_bioactivity_dataset[~chembl_papyrus_bioactivity_dataset['target_id'].str.contains('WT')].groupby\
    (['source']).count()

#### Get an example assay for Figure 1 of the MS
Where two possible mutations were extracted, one of them being validated in the end and the other one being defined
as an exception

In [ ]:
# Visualize assays with 2 exceptions defined and no previously ChEMBL annotated variants
exceptions[exceptions.apply(lambda x: len(x['exception_reasons']) == 2, axis=1) & exceptions["mutation"].isnull()]

In [ ]:
# Manually selected assay ID for Figure 1`based on options above
fig1_assay_id = 306767

In [ ]:
# Assay description
print(str(exceptions[exceptions["assay_id"] == fig1_assay_id].description.tolist()[0]))

In [ ]:
# Assay data
exceptions[exceptions["assay_id"] == fig1_assay_id]

In [ ]:
# Bioactivity data available for the mutant in the defiend assay
chembl_bioactivity_dataset[chembl_bioactivity_dataset["target_id"] == "P47900_Y306F"]

#### Annotation statistics

In [ ]:
# Check for how many assay-accession pairs we have extracted and validated mutants
chembl_mutants_extracted = chembl_assays_annotated[chembl_assays_annotated.apply(lambda x: x['Protein_Type'] != 'WT', axis=1)]
chembl_mutants_extracted

In [ ]:
# Check how many of the extracted mutants were not previously defined in ChEMBL's variant variable 'mutation'
chembl_new_mutants_extracted1 = chembl_mutants_extracted[(chembl_mutants_extracted['mutation'].isna())]
chembl_new_mutants_extracted1

In [ ]:
# Check how many of the extracted mutants were defined as mutated by the actual mutation was not specified on ChEMBL
chembl_new_mutants_extracted2 = chembl_mutants_extracted[chembl_mutants_extracted['mutation'] == 'UNDEFINED MUTATION']
chembl_new_mutants_extracted2

In [ ]:
# Check number of assays per target if target is defined as 'accession' AND 'Protein_Type' (i.e. 'target_id')
chembl_assays_annotated.groupby(['accession','Protein_Type'])['assay_id'].count()

In [ ]:
# Check number of assays per target if target is defined as 'Protein_Type'
# i.e. all WT proteins together and every mutant separately
chembl_assays_annotated.groupby(['Protein_Type'])['assay_id'].count()

In [ ]:
# Check number of assays per target if target is defined as 'accession'
chembl_assays_annotated.groupby(['accession'])['assay_id'].count()